# Packages

In [35]:
import pandas as pd
from geopy.geocoders import Nominatim
import numpy as np
import googlemaps

In [36]:
folder_path = "C:\KU Leuven\Modern Data Analytics - MDA\MDA Project\MDA"
# pit_loc = pd.read_parquet(f"{folder_path}/ambulance_locations.parquet.gzip", engine="pyarrow")
# amb_loc = pd.read_parquet(f"{folder_path}/mug_locations.parquet.gzip", engine="pyarrow")
# mug_loc = pd.read_parquet(f"{folder_path}/pit_locations.parquet.gzip", engine="pyarrow")
aed_loc = pd.read_parquet(f"{folder_path}\dataset\\aed_locations.parquet.gzip", engine="pyarrow")

# AED Dataset 

In [37]:
aed_loc.shape

(15227, 11)

In [38]:
aed_loc.head()

,id,type,address,number,postal_code,municipality,province,location,public,available,hours
0,13.0,None,Blvd. Fr. Roosevelt,24.0,7060.0,SOIGNIES,Hainaut,None,Y,None,None
1,70.0,None,Ch. De Wégimont,76.0,4630.0,Ayeneux,Liège,None,None,None,None
2,71.0,None,Place Saint - Lambert,NaN,4020.0,Liège,Liège,None,None,None,None
3,72.0,None,Rue du Doyard,NaN,4990.0,Lierneux,Liège,None,None,None,None
4,73.0,None,Fond Saint Servais,NaN,4000.0,Liège,Liège,None,None,None,None


## Clean the Dataset 

Identify Missing Values

In [39]:
# Inspect NA values
aed_loc.isna().sum() / aed_loc.shape[0]

id              0.000131
type            0.660669
address         0.000131
number          0.140671
postal_code     0.002233
municipality    0.005582
province        0.002430
location        0.446838
public          0.469429
available       0.718264
hours           0.929270
dtype: float64

Drop Type, Location, Public, Available and Hours since they contain a significant amount of missing values

In [40]:
#Drop columns with high NA values
aed_loc.drop(["type","location","public","available","hours"], axis=1, inplace=True)

# Drop NAs
aed_loc.dropna(subset=['number'],inplace=True)

# Drop NAs
aed_loc.dropna(subset=['municipality'], inplace=True)

# Drop NAs
aed_loc.dropna(inplace=True)

# Convert number and postal code to integer and then string 
aed_loc["number"] = aed_loc["number"].astype(int).astype(str)
aed_loc["postal_code"] = aed_loc["postal_code"].astype(int).astype(str)



Make the string type columns similar to each other using lower and strip

In [41]:
def lower_strip(x):
    return x.lower().strip(" ")

for name in ["address","municipality","province"]:
    aed_loc[name] = aed_loc[name].apply(lambda x: lower_strip(x))


In [42]:
aed_loc.head()

,id,address,number,postal_code,municipality,province
0,13.0,blvd. fr. roosevelt,24,7060,soignies,hainaut
1,70.0,ch. de wégimont,76,4630,ayeneux,liège
5,74.0,rue des prémontrès,12,4020,liège,liège
6,75.0,route de bastogne,1,4920,harzé,liège
7,76.0,rue du parc,1,4540,jehay,liège


## Explore and Drop Duplicate Rows 

Check for duplicates

In [43]:
aed_loc[aed_loc.duplicated(["address","number","postal_code","municipality","province"])]

,id,address,number,postal_code,municipality,province
139,453.0,nijverheidslaan,7,3200,aarschot,vlaams-brabant
142,456.0,nijverheidslaan,7,3200,aarschot,vlaams-brabant
181,521.0,boomsestwg.,957,2610,wilrijk,antwerpen
234,613.0,st. pietersnieuwstraat,23,9000,gent,oost-vlaanderen
258,741.0,nijverheidsstraat,15,2260,oevel,antwerpen
...,...,...,...,...,...,...
15214,14575.0,carrefour de la spetz,2,6700,arlon,luxembourg
15219,16659.0,rue lanoy,52,7740,pecq,hainaut
15221,16661.0,nekkerspoel-borcht,19,2800,mechelen,antwerpen
15222,16662.0,nekkerspoel-borcht,19,2800,mechelen,antwerpen


In [44]:
aed_loc[aed_loc["address"]=="nijverheidslaan"]

,id,address,number,postal_code,municipality,province
138,452.0,nijverheidslaan,7,3200,aarschot,vlaams-brabant
139,453.0,nijverheidslaan,7,3200,aarschot,vlaams-brabant
142,456.0,nijverheidslaan,7,3200,aarschot,vlaams-brabant
4562,5641.0,nijverheidslaan,11,8970,poperinge,west-vlaanderen
6275,7534.0,nijverheidslaan,9,8570,vichte,west-vlaanderen
6695,7996.0,nijverheidslaan,3,9880,aalter,oost-vlaanderen
7010,8345.0,nijverheidslaan,52,8560,wevelgem,west-vlaanderen
7241,8606.0,nijverheidslaan,24,3290,diest,vlaams-brabant
8732,10177.0,nijverheidslaan,6,8890,moorslede,west-vlaanderen
11031,12774.0,nijverheidslaan,19,8580,avelgem,west-vlaanderen


Drop Duplicates

In [45]:
aed_loc.drop_duplicates(["address","number","postal_code","municipality","province"], inplace=True)

Check for Duplicates

In [46]:
aed_loc[aed_loc.duplicated(["address","number","postal_code","province"])]


,id,address,number,postal_code,municipality,province
730,1388.0,place victor horta,11,1060,brussel,bruxelles-brussel
1302,2097.0,rue aily,1,4431,ans,liège
2233,3117.0,rue a. materne,80,4460,bierset,liège
2322,3214.0,domeinstraat,61,3010,kessel-lo,vlaams-brabant
2766,3694.0,hermeslaan,2,1831,machelen,vlaams-brabant
...,...,...,...,...,...,...
15127,16569.0,siemenslaan,1,8020,hertsberge,west-vlaanderen
15162,16602.0,colmanstraat,51,9270,laarne,oost-vlaanderen
15163,16603.0,keistraat,5,9270,kalken,oost-vlaanderen
15192,16629.0,avenue du boulevard,21,1210,saint-josse-ten-noode,bruxelles-brussel


In [47]:
aed_loc.shape

(11225, 6)

In [48]:
aed_loc[aed_loc["address"]=="place victor horta"]

,id,address,number,postal_code,municipality,province
638,1265.0,place victor horta,40,1060,bruxelles,bruxelles-brussel
729,1387.0,place victor horta,11,1060,bruxelles,bruxelles-brussel
730,1388.0,place victor horta,11,1060,brussel,bruxelles-brussel
11636,13633.0,place victor horta,11,1060,saint-gilles,bruxelles-brussel


In [49]:
aed_loc.drop_duplicates(["address","number","postal_code","province"], inplace=True)

In [50]:
aed_loc[aed_loc.duplicated(["address","number","postal_code","municipality"])]

,id,address,number,postal_code,municipality,province
8979,10457.0,jan olieslagerslaan,35,1800,vilvoorde,limburg
11521,13502.0,rootenstraat,8,3600,genk,oost-vlaanderen
13518,15458.0,schaliehoevestraat,12,2220,heist-op-den-berg,vlaams-brabant
13636,15577.0,brownfieldlaan,19,2830,willebroek,antwerpen
14082,15870.0,ilgatlaan,23,3500,hasselt,west-vlaanderen
14813,16273.0,herentalsesteenweg,64,2460,kasterlee,luxembourg
14821,16278.0,sterrebeekstraat,172,1930,zaventem,bruxelles-brussel


In [51]:
aed_loc.shape

(11105, 6)

In [52]:
aed_loc[aed_loc["address"]=="jan olieslagerslaan"]

,id,address,number,postal_code,municipality,province
7228,8592.0,jan olieslagerslaan,35,1800,vilvoorde,vlaams-brabant
8979,10457.0,jan olieslagerslaan,35,1800,vilvoorde,limburg
12236,14261.0,jan olieslagerslaan,41,1800,vilvoorde,vlaams-brabant


In [53]:
aed_loc.drop_duplicates(["address","number","postal_code","municipality"], inplace=True)

In [54]:
aed_loc[aed_loc.duplicated(["address","number","postal_code"])]

,id,address,number,postal_code,municipality,province
12493,14560.0,boulevard du souverain,165,1160,auderghem,vlaams-brabant


In [55]:
aed_loc[aed_loc["address"]=="boulevard du souverain"]

,id,address,number,postal_code,municipality,province
3132,4111.0,boulevard du souverain,68,1170,bruxelles,bruxelles-brussel
3657,4671.0,boulevard du souverain,36,1170,bruxelles,bruxelles-brussel
9282,10772.0,boulevard du souverain,183,1160,bruxelles,bruxelles-brussel
12358,14414.0,boulevard du souverain,165,1160,oudergem,bruxelles-brussel
12493,14560.0,boulevard du souverain,165,1160,auderghem,vlaams-brabant
13647,15586.0,boulevard du souverain,25,1170,watermael-boitsfort,bruxelles-brussel
15002,16451.0,boulevard du souverain,24,1170,bruxelles,bruxelles-brussel


In [56]:
aed_loc.drop_duplicates(["address","number","postal_code"], inplace=True)

In [57]:
aed_loc[aed_loc.duplicated(["id","address","number"])]

,id,address,number,postal_code,municipality,province
3474,4470.0,verenigde-natieslaan,1,9000,gent,oost-vlaanderen


In [58]:
aed_loc[aed_loc["id"]==4470.0]

,id,address,number,postal_code,municipality,province
3473,4470.0,verenigde-natieslaan,1,900,gent,oost-vlaanderen
3474,4470.0,verenigde-natieslaan,1,9000,gent,oost-vlaanderen


In [59]:
aed_loc.drop_duplicates(["id","address","number"], inplace=True)

In [60]:
aed_loc[aed_loc.duplicated(["id","number","province"])]

,id,address,number,postal_code,municipality,province
13164,3372.0,tussen ‘t pas,9,9000,gent,oost-vlaanderen


In [61]:
aed_loc[aed_loc["id"]==3372.0]

,id,address,number,postal_code,municipality,province
2468,3372.0,tussen t pas,9,9000,gent,oost-vlaanderen
13164,3372.0,tussen ‘t pas,9,9000,gent,oost-vlaanderen


In [62]:
aed_loc.drop_duplicates(["id","number","province"], inplace=True)

In [63]:
aed_loc.shape

(11095, 6)

In [64]:
aed_loc["province"].unique()

array(['hainaut', 'liège', 'bruxelles-brussel', 'vlaams-brabant',
       'luxembourg', 'antwerpen', 'limburg', 'west-vlaanderen',
       'oost-vlaanderen', 'brabant wallon', 'namur'], dtype=object)

In [65]:
aed_loc.head()

,id,address,number,postal_code,municipality,province
0,13.0,blvd. fr. roosevelt,24,7060,soignies,hainaut
1,70.0,ch. de wégimont,76,4630,ayeneux,liège
5,74.0,rue des prémontrès,12,4020,liège,liège
6,75.0,route de bastogne,1,4920,harzé,liège
7,76.0,rue du parc,1,4540,jehay,liège


## Google Maps API

- Obtain coordinates of the AED locations
- Explore and drop similar rows in terms of coordinates 

Create new column that has all the location information to be used by the API

In [489]:
# Create a new column containing information on number,postal code and province of an address 
aed_loc['unique_address'] = aed_loc['address'].str.cat(others=aed_loc[["number","postal_code","municipality","province"]], sep=',',na_rep='')

In [490]:
aed_loc.head()

,id,address,number,postal_code,municipality,province,unique_address
0,13.0,blvd. fr. roosevelt,24,7060,soignies,hainaut,"blvd. fr. roosevelt,24,7060,soignies,hainaut"
1,70.0,ch. de wégimont,76,4630,ayeneux,liège,"ch. de wégimont,76,4630,ayeneux,liège"
5,74.0,rue des prémontrès,12,4020,liège,liège,"rue des prémontrès,12,4020,liège,liège"
6,75.0,route de bastogne,1,4920,harzé,liège,"route de bastogne,1,4920,harzé,liège"
7,76.0,rue du parc,1,4540,jehay,liège,"rue du parc,1,4540,jehay,liège"


In [491]:
aed_loc.dtypes

id                float64
address            object
number             object
postal_code        object
municipality       object
province           object
unique_address     object
dtype: object

In [492]:
aed_loc.shape

(11095, 7)

In [494]:
aed_loc.head()

,id,address,number,postal_code,municipality,province,unique_address
0,13.0,blvd. fr. roosevelt,24,7060,soignies,hainaut,"blvd. fr. roosevelt,24,7060,soignies,hainaut"
1,70.0,ch. de wégimont,76,4630,ayeneux,liège,"ch. de wégimont,76,4630,ayeneux,liège"
5,74.0,rue des prémontrès,12,4020,liège,liège,"rue des prémontrès,12,4020,liège,liège"
6,75.0,route de bastogne,1,4920,harzé,liège,"route de bastogne,1,4920,harzé,liège"
7,76.0,rue du parc,1,4540,jehay,liège,"rue du parc,1,4540,jehay,liège"


Define the function that utilizes the Google Maps API to transform location information into coordinates

In [302]:
gmaps_key = googlemaps.Client(key="")

def geocode(add):
    g = gmaps_key.geocode(add)
    if g is not None and isinstance(g, list) and len(g) > 0:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lng, lat)
    else:
        return None


In [303]:
aed_loc.head()

,id,address,number,postal_code,municipality,province,unique_address
0,13.0,blvd. fr. roosevelt,24,7060,soignies,hainaut,"blvd. fr. roosevelt,24,7060,soignies,hainaut"
1,70.0,ch. de wégimont,76,4630,ayeneux,liège,"ch. de wégimont,76,4630,ayeneux,liège"
5,74.0,rue des prémontrès,12,4020,liège,liège,"rue des prémontrès,12,4020,liège,liège"
6,75.0,route de bastogne,1,4920,harzé,liège,"route de bastogne,1,4920,harzé,liège"
7,76.0,rue du parc,1,4540,jehay,liège,"rue du parc,1,4540,jehay,liège"


In [304]:
aed_loc.shape

(11095, 7)

Apply the function to convert location information to coordinates

In [305]:
aed_loc["points_coordinates"] = aed_loc["unique_address"].apply(geocode)

In [318]:
aed_loc.head()

,id,address,number,postal_code,municipality,province,unique_address,points_coordinates
0,13.0,blvd. fr. roosevelt,24,7060,soignies,hainaut,"blvd. fr. roosevelt,24,7060,soignies,hainaut","(4.0655019, 50.5767646)"
1,70.0,ch. de wégimont,76,4630,ayeneux,liège,"ch. de wégimont,76,4630,ayeneux,liège","(5.7397828, 50.6105697)"
5,74.0,rue des prémontrès,12,4020,liège,liège,"rue des prémontrès,12,4020,liège,liège","(5.5739754, 50.6380181)"
6,75.0,route de bastogne,1,4920,harzé,liège,"route de bastogne,1,4920,harzé,liège","(5.6658992, 50.4411102)"
7,76.0,rue du parc,1,4540,jehay,liège,"rue du parc,1,4540,jehay,liège","(5.3231573, 50.5772875)"


In [324]:
aed_loc.shape

(10627, 8)

In [320]:
aed_loc.isna().sum()

id                    0
address               0
number                0
postal_code           0
municipality          0
province              0
unique_address        0
points_coordinates    0
dtype: int64

## Explore and Drop rows with same coordinates

- Drop rows have correspond to the same coordinates and thus to the same location 

In [321]:
aed_loc[aed_loc.duplicated(["points_coordinates"])]

,id,address,number,postal_code,municipality,province,unique_address,points_coordinates


In [323]:
aed_loc.drop_duplicates(["points_coordinates"],inplace=True)

In [347]:
aed_loc.shape

(10627, 8)

Define function to extrac the lat and lng from the coordinates column

In [348]:
def lat_from_tuple(x):
    return x[1]

def lng_from_tuple(x):
    return x[0]

In [349]:
aed_loc["lat"] = aed_loc["points_coordinates"].apply(lat_from_tuple)
aed_loc["lng"] = aed_loc["points_coordinates"].apply(lng_from_tuple)

In [361]:
aed_loc["new_lat"] = aed_loc["lat"].round(6)
aed_loc["new_lng"] = aed_loc["lng"].round(6)

In [381]:
# pd.set_option('display.float_format', '{:.7f}'.format)
# pd.reset_option('display.float_format')
aed_loc.head()

,id,address,number,postal_code,municipality,province,unique_address,points_coordinates,lat,lng
0,13.0,blvd. fr. roosevelt,24,7060,soignies,hainaut,"blvd. fr. roosevelt,24,7060,soignies,hainaut","(4.0655019, 50.5767646)",50.576765,4.065502
1,70.0,ch. de wégimont,76,4630,ayeneux,liège,"ch. de wégimont,76,4630,ayeneux,liège","(5.7397828, 50.6105697)",50.610570,5.739783
5,74.0,rue des prémontrès,12,4020,liège,liège,"rue des prémontrès,12,4020,liège,liège","(5.5739754, 50.6380181)",50.638018,5.573975
6,75.0,route de bastogne,1,4920,harzé,liège,"route de bastogne,1,4920,harzé,liège","(5.6658992, 50.4411102)",50.441110,5.665899
7,76.0,rue du parc,1,4540,jehay,liège,"rue du parc,1,4540,jehay,liège","(5.3231573, 50.5772875)",50.577287,5.323157


In [367]:
aed_loc[aed_loc.duplicated(["lat","lng"])]

,id,address,number,postal_code,municipality,province,unique_address,points_coordinates,lat,lng,new_lat,new_lng


In [359]:
aed_loc.drop_duplicates(["new_lat","new_lng","number"], inplace=True)

In [383]:
aed_loc.isna().sum()

id                    0
address               0
number                0
postal_code           0
municipality          0
province              0
unique_address        0
points_coordinates    0
lat                   0
lng                   0
dtype: int64

In [370]:
aed_loc.drop(["new_lat","new_lng"],axis=1,inplace=True)

In [459]:
aed_loc.shape

(11095, 7)

In [458]:
aed_loc.head()

,id,address,number,postal_code,municipality,province,unique_address
0,13.0,blvd. fr. roosevelt,24,7060,soignies,hainaut,"blvd. fr. roosevelt,24,7060,soignies,hainaut"
1,70.0,ch. de wégimont,76,4630,ayeneux,liège,"ch. de wégimont,76,4630,ayeneux,liège"
5,74.0,rue des prémontrès,12,4020,liège,liège,"rue des prémontrès,12,4020,liège,liège"
6,75.0,route de bastogne,1,4920,harzé,liège,"route de bastogne,1,4920,harzé,liège"
7,76.0,rue du parc,1,4540,jehay,liège,"rue du parc,1,4540,jehay,liège"


## Save the dataset as csv file

- The file AED_coordinates_NEWEST_2 contains all the AED and their coordinates
- Few of the AED in the dataset are not within Belgium. This will be address in a different notebook

In [391]:
aed_loc.to_csv('C:\KU Leuven\Modern Data Analytics - MDA\MDA Project\AED_coordinates_NEWEST_2.csv', index=False)

## Read the stored coordinates dataset from the pc (AED_coordinates)

In [66]:
aed_coord = pd.read_csv(r"C:\KU Leuven\Modern Data Analytics - MDA\MDA Project BackUp\AED_coordinates_NEWEST_2.csv", index_col=False)


In [67]:
aed_coord.shape

(10626, 10)

In [497]:
aed_coord.isna().sum()

id                    0
address               0
number                0
postal_code           0
municipality          0
province              0
unique_address        0
points_coordinates    0
lat                   0
lng                   0
dtype: int64

In [496]:
aed_coord.head()

,id,address,number,postal_code,municipality,province,unique_address,points_coordinates,lat,lng
0,13.0,blvd. fr. roosevelt,24,7060,soignies,hainaut,"blvd. fr. roosevelt,24,7060,soignies,hainaut","(4.0655019, 50.5767646)",50.576765,4.065502
1,70.0,ch. de wégimont,76,4630,ayeneux,liège,"ch. de wégimont,76,4630,ayeneux,liège","(5.7397828, 50.6105697)",50.610570,5.739783
2,74.0,rue des prémontrès,12,4020,liège,liège,"rue des prémontrès,12,4020,liège,liège","(5.5739754, 50.6380181)",50.638018,5.573975
3,75.0,route de bastogne,1,4920,harzé,liège,"route de bastogne,1,4920,harzé,liège","(5.6658992, 50.4411102)",50.441110,5.665899
4,76.0,rue du parc,1,4540,jehay,liège,"rue du parc,1,4540,jehay,liège","(5.3231573, 50.5772875)",50.577287,5.323157


In [ ]:
# SAVE TO COMPUTER !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
# SAVE TO COMPUTER !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# AMBULANCE Dataset

The ambulance locations dataset already contains the coordinates thus no need to use the Google Maps API

In [318]:
amb_loc = pd.read_parquet(f"{folder_path}/ambulance_locations.parquet.gzip", engine="pyarrow")

In [319]:
amb_loc.head()

,base,medical_resource,province,region,departure_location,departure_location_number,latitude,longitude,occasional_permanence
0,BA ANTW [Borgerhout] AMBUCE,ZW ANTWERPEN 7 (Team: AAANTW07A),Antwerpen,Vlaams Gewest,Gijselstraat 150 2140 BORGERHOUT,3811,51.215516,4.443788,N
1,BA ANTW [Hoboken] AMBUCE,ZW ANTWERPEN 9 (Team: AAANTW09A),Antwerpen,Vlaams Gewest,Antwerpsesteenweg 242 2660 HOBOKEN,3812,51.178831,4.361948,N
2,BA ANTW [Hof Ter Schelde] AC Antwerpen,ZW ANTWERPEN 3 (Team: AAANTW03A),Antwerpen,Vlaams Gewest,August Vermeylenlaan 6 2050 ANTWERPEN,10902,51.232810,4.377237,N
3,BA ANTW [Kaai 606] SIHWA,ZW ANTWERPEN HAVEN 1 (Team: AAAPOR01A)_x000D_Z...,Antwerpen,Vlaams Gewest,Antwerpsebaan 4 (Kaai 606) 2040 ANTWERPEN,12902,51.315516,4.321506,N
4,BA ANTW [Sint Elisabeth] AC Antwerpen,ZW ANTWERPEN 2 (Team: AAANTW02A),Antwerpen,Vlaams Gewest,Leopoldstraat 26 2000 ANTWERPEN,10901,51.213623,4.405718,N


In [320]:
amb_loc.shape

(279, 9)

In [321]:
amb_loc.isna().sum() / amb_loc.shape[0]

base                         0.000000
medical_resource             0.000000
province                     0.046595
region                       0.000000
departure_location           0.000000
departure_location_number    0.000000
latitude                     0.000000
longitude                    0.000000
occasional_permanence        0.000000
dtype: float64

# MUG Dataset 

## Clean Dataset

- Identify NA values
- Drop the name campus column 

In [ ]:
C:\KU Leuven\Modern Data Analytics - MDA\MDA Project\MDA\dataset\mug_locations.parquet.gzip

In [69]:
mug_loc = pd.read_parquet(f"{folder_path}\dataset\mug_locations.parquet.gzip", engine="pyarrow")

In [100]:
mug_loc.head()

,hospital_id,mug_id,campus_id,name_hospital,name_campus,address_campus,postal_code,municipality,region,province
0,9,102000,6230,ZIEKENHUISNETWERK ANTWERPEN (ZNA),ZNA CADIX,KEMPENSTRAAT 100,2030,ANTWERPEN,V,Antwerpen
1,99,102000,2020,GZA- ZIEKENHUIZEN,SINT-VINCENTIUS,ST.-VINCENTIUSSTRAAT 20,2018,ANTWERPEN,V,Antwerpen
2,682,102000,1210,AZ MONICA,DEURNE,FLORENT PAUWELSLEI 1,2100,DEURNE,V,Antwerpen
3,9,103000,2000,ZIEKENHUISNETWERK ANTWERPEN (ZNA),ZNA MIDDELHEIM,LINDENDREEF 1,2020,ANTWERPEN,V,Antwerpen
4,99,103000,1290,GZA- ZIEKENHUIZEN,SINT- AUGUSTINUS,OOSTERVELDLAAN 24,2610,WILRIJK,V,Antwerpen


In [70]:
mug_loc.shape

(94, 10)

In [166]:
mug_loc.isna().sum()

hospital_id        0
mug_id             0
campus_id          0
name_hospital      0
name_campus       35
address_campus     0
postal_code        0
municipality       0
region             0
province           0
dtype: int64

In [167]:
mug_loc.drop("name_campus", axis=1, inplace=True)

In [171]:
mug_loc[mug_loc.duplicated()]

,hospital_id,mug_id,campus_id,name_hospital,address_campus,postal_code,municipality,region,province


In [172]:
mug_loc.dtypes

hospital_id        int64
mug_id             int64
campus_id          int64
name_hospital     object
address_campus    object
postal_code        int64
municipality      object
region            object
province          object
dtype: object

In [177]:
mug_loc["postal_code"] = mug_loc["postal_code"].astype(str)

In [180]:
# Create a new column containing information on number,postal code and province of an address 
mug_loc['unique_address'] = mug_loc['address_campus'].str.cat(others=mug_loc[["municipality","postal_code","province"]], sep=',',na_rep='')

In [181]:
mug_loc.head()

,hospital_id,mug_id,campus_id,name_hospital,address_campus,postal_code,municipality,region,province,unique_address
0,9,102000,6230,ZIEKENHUISNETWERK ANTWERPEN (ZNA),KEMPENSTRAAT 100,2030,ANTWERPEN,V,Antwerpen,"KEMPENSTRAAT 100,ANTWERPEN,2030,Antwerpen"
1,99,102000,2020,GZA- ZIEKENHUIZEN,ST.-VINCENTIUSSTRAAT 20,2018,ANTWERPEN,V,Antwerpen,"ST.-VINCENTIUSSTRAAT 20,ANTWERPEN,2018,Antwerpen"
2,682,102000,1210,AZ MONICA,FLORENT PAUWELSLEI 1,2100,DEURNE,V,Antwerpen,"FLORENT PAUWELSLEI 1,DEURNE,2100,Antwerpen"
3,9,103000,2000,ZIEKENHUISNETWERK ANTWERPEN (ZNA),LINDENDREEF 1,2020,ANTWERPEN,V,Antwerpen,"LINDENDREEF 1,ANTWERPEN,2020,Antwerpen"
4,99,103000,1290,GZA- ZIEKENHUIZEN,OOSTERVELDLAAN 24,2610,WILRIJK,V,Antwerpen,"OOSTERVELDLAAN 24,WILRIJK,2610,Antwerpen"


In [182]:
mug_loc["mug_coordinates"] = mug_loc["unique_address"].apply(geocode)

In [184]:
mug_loc.isna().sum()

hospital_id        0
mug_id             0
campus_id          0
name_hospital      0
address_campus     0
postal_code        0
municipality       0
region             0
province           0
unique_address     0
mug_coordinates    0
dtype: int64

In [188]:
mug_loc.shape

(94, 11)

## Save and read the dataset

In [3]:
mug_loc.to_csv('C:\KU Leuven\Modern Data Analytics - MDA\MDA Project\MUG_coordinates_new.csv', index=False)

NameError: name 'mug_loc' is not defined

In [4]:
mug_coord = pd.read_csv(r'C:\KU Leuven\Modern Data Analytics - MDA\MDA Project\MUG_coordinates_new.csv', index_col=False)

In [187]:
mug_coord.isna().sum()

hospital_id        0
mug_id             0
campus_id          0
name_hospital      0
address_campus     0
postal_code        0
municipality       0
region             0
province           0
unique_address     0
mug_coordinates    0
dtype: int64

In [189]:
mug_coord.shape

(94, 11)

In [5]:
mug_coord.head()

,hospital_id,mug_id,campus_id,name_hospital,address_campus,postal_code,municipality,region,province,unique_address,mug_coordinates
0,9,102000,6230,ZIEKENHUISNETWERK ANTWERPEN (ZNA),KEMPENSTRAAT 100,2030,ANTWERPEN,V,Antwerpen,"KEMPENSTRAAT 100,ANTWERPEN,2030,Antwerpen","(51.2313796, 4.415252)"
1,99,102000,2020,GZA- ZIEKENHUIZEN,ST.-VINCENTIUSSTRAAT 20,2018,ANTWERPEN,V,Antwerpen,"ST.-VINCENTIUSSTRAAT 20,ANTWERPEN,2018,Antwerpen","(51.2060094, 4.4126797)"
2,682,102000,1210,AZ MONICA,FLORENT PAUWELSLEI 1,2100,DEURNE,V,Antwerpen,"FLORENT PAUWELSLEI 1,DEURNE,2100,Antwerpen","(51.2063667, 4.4702018)"
3,9,103000,2000,ZIEKENHUISNETWERK ANTWERPEN (ZNA),LINDENDREEF 1,2020,ANTWERPEN,V,Antwerpen,"LINDENDREEF 1,ANTWERPEN,2020,Antwerpen","(51.183318, 4.4215846)"
4,99,103000,1290,GZA- ZIEKENHUIZEN,OOSTERVELDLAAN 24,2610,WILRIJK,V,Antwerpen,"OOSTERVELDLAAN 24,WILRIJK,2610,Antwerpen","(51.17522349999999, 4.4166534)"


## REMARK !!! THE MUG Dataset needs to be explored for duplicates 

# PIT Dataset

- Drop NA values (only 3 NA)
- Create column with location information of the hospital the mug is located
- Identify the hospital (and thus mug) coordinates using Google Maps API

In [97]:
pit_loc = pd.read_parquet(f"{folder_path}/pit_locations.parquet.gzip", engine="pyarrow")

In [98]:
pit_loc.head()

,unit,campus,province,region,unit_id,ambucode,ambusitecode,link
0,"100001: PIT Rumst, AZ Rivierland",Ziekenhuis/Hôpital: 104 - AZ RIVIERENLAND --- ...,Antwerpen/Anvers,Vlaanderen/Flandre,PARUMS01A,191,19101,None
1,"102001: PIT Deinze, St Vincentius",Ziekenhuis/Hôpital: 134 - AZ SINT-VINCENTIUS -...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,PODEIN01A,413,41301,None
2,102101: PIT Stuivenberg,Ziekenhuis/Hôpital: 009 - ZIEKENHUISNETWERK AN...,Antwerpen/Anvers,Vlaanderen/Flandre,PAANTW01A,117,11704,None
3,"110001: PIT Lier, Heilig Hart",Ziekenhuis/Hôpital: 097 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PALIER01A,171,17101,None
4,113001: PIT HEILIG HART ZIEKENHUIS,Ziekenhuis/Hôpital: 102 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PAMOL_01A,190,19001,None


In [120]:
pit_loc.isna().sum()

unit            0
campus          0
province        3
region          0
unit_id         0
ambucode        0
ambusitecode    0
dtype: int64

In [119]:
pit_loc.drop("link", axis=1, inplace=True)

In [121]:
pit_loc.shape

(24, 7)

In [126]:
geocode('AZ RIVIERENLAND')

In [112]:
pit_loc["campus"][0].split("---")

['Ziekenhuis/Hôpital: 104 - AZ RIVIERENLAND ', ' Campus/Site: 1270 - RUMST']

In [135]:
pit_loc["campus"][0].split("---")[1].split("-")[1].strip()

'RUMST'

In [130]:
print(pit_loc["campus"].tolist())

['Ziekenhuis/Hôpital: 104 - AZ RIVIERENLAND --- Campus/Site: 1270 - RUMST', 'Ziekenhuis/Hôpital: 134 - AZ SINT-VINCENTIUS --- Campus/Site: 1830 - AZ SINT-VINCENTIUS DEINZE', 'Ziekenhuis/Hôpital: 009 - ZIEKENHUISNETWERK ANTWERPEN (ZNA) --- Campus/Site: 1180 - ZNA STUIVENBERG', 'Ziekenhuis/Hôpital: 097 - HEILIG HART ZIEKENHUIS --- Campus/Site: 1260 - HEILIG HART', 'Ziekenhuis/Hôpital: 102 - HEILIG HART ZIEKENHUIS --- Campus/Site: 2080 - HEILIG HART ZIEKENHUIS MOL', 'Ziekenhuis/Hôpital: 595 - VITAZ --- Campus/Site: 1650 - VITAZ CAMPUS LOKEREN', 'Ziekenhuis/Hôpital: 126 - ONZE-LIEVE-VROUW ZIEKENHUIS --- Campus/Site: 1300 - OLV ZIEKENHUIS ASSE', 'Ziekenhuis/Hôpital: 049 - AZ SINT-JAN BRUGGE - OOSTENDE --- Campus/Site: 1010 - AZ SINT-JAN BRUGGE', 'Ziekenhuis/Hôpital: 140 - AZ SINT-LUCAS --- Campus/Site: 1000 - SINT- LUCAS', 'Ziekenhuis/Hôpital: 392 - AZ ZENO --- Campus/Site: 1040 - AZ ZENO BLANKENBERGE', 'Ziekenhuis/Hôpital: 371 - ZIEKENHUIS OOST-LIMBURG --- Campus/Site: 3560 - SINT-JAN GENK

In [136]:
def find_hospital(text):
    hospital = text.split("---")[0].split('-')[1].strip()
    region = text.split("---")[1].split("-")[1].strip()
    hospital_region = hospital + " " + region
    return hospital_region


In [137]:
find_hospital(pit_loc["campus"][0])

'AZ RIVIERENLAND RUMST'

In [138]:
pit_loc["pit_hospital"] = pit_loc["campus"].apply(find_hospital)

In [139]:
pit_loc

,unit,campus,province,region,unit_id,ambucode,ambusitecode,pit_hospital,pit_hospital_coordinates
0,"100001: PIT Rumst, AZ Rivierland",Ziekenhuis/Hôpital: 104 - AZ RIVIERENLAND --- ...,Antwerpen/Anvers,Vlaanderen/Flandre,PARUMS01A,191,19101,AZ RIVIERENLAND RUMST,None
1,"102001: PIT Deinze, St Vincentius",Ziekenhuis/Hôpital: 134 - AZ SINT-VINCENTIUS -...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,PODEIN01A,413,41301,AZ SINT AZ SINT,"(34.0489281, -111.0937311)"
2,102101: PIT Stuivenberg,Ziekenhuis/Hôpital: 009 - ZIEKENHUISNETWERK AN...,Antwerpen/Anvers,Vlaanderen/Flandre,PAANTW01A,117,11704,ZIEKENHUISNETWERK ANTWERPEN (ZNA) ZNA STUIVENBERG,"(51.1817753, 4.4212415)"
3,"110001: PIT Lier, Heilig Hart",Ziekenhuis/Hôpital: 097 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PALIER01A,171,17101,HEILIG HART ZIEKENHUIS HEILIG HART,None
4,113001: PIT HEILIG HART ZIEKENHUIS,Ziekenhuis/Hôpital: 102 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PAMOL_01A,190,19001,HEILIG HART ZIEKENHUIS HEILIG HART ZIEKENHUIS MOL,None
5,200001: PIT LOKEREN,Ziekenhuis/Hôpital: 595 - VITAZ --- Campus/Sit...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,POLOKE01A,440,44001,VITAZ VITAZ CAMPUS LOKEREN,"(48.97116519999999, 20.9546432)"
6,"210001: PIT ASSE, OLV",Ziekenhuis/Hôpital: 126 - ONZE-LIEVE-VROUW ZIE...,Vlaams Brabant/Brabant flamand,Vlaanderen/Flandre,PVASSE01A,480,48001,ONZE OLV ZIEKENHUIS ASSE,None
7,301001: PIT ST Jan (Associatie St Lucas),Ziekenhuis/Hôpital: 049 - AZ SINT-JAN BRUGGE -...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBRUG01A,391,39102,AZ SINT AZ SINT,"(34.0489281, -111.0937311)"
8,301001: PIT St Lucas Brugge (associatie St Jan),Ziekenhuis/Hôpital: 140 - AZ SINT-LUCAS --- Ca...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBRUG02A,391,39101,AZ SINT SINT,"(34.0489281, -111.0937311)"
9,303001: PIT Blankenberge,Ziekenhuis/Hôpital: 392 - AZ ZENO --- Campus/S...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBLAN01A,374,37401,AZ ZENO AZ ZENO BLANKENBERGE,"(51.327595, 3.2912258)"


In [140]:
pit_loc["pit_hospital_coordinates"]=pit_loc["pit_hospital"].apply(geocode)

In [141]:
pit_loc

,unit,campus,province,region,unit_id,ambucode,ambusitecode,pit_hospital,pit_hospital_coordinates
0,"100001: PIT Rumst, AZ Rivierland",Ziekenhuis/Hôpital: 104 - AZ RIVIERENLAND --- ...,Antwerpen/Anvers,Vlaanderen/Flandre,PARUMS01A,191,19101,AZ RIVIERENLAND RUMST,"(51.1064835, 4.371312)"
1,"102001: PIT Deinze, St Vincentius",Ziekenhuis/Hôpital: 134 - AZ SINT-VINCENTIUS -...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,PODEIN01A,413,41301,AZ SINT AZ SINT,"(34.0489281, -111.0937311)"
2,102101: PIT Stuivenberg,Ziekenhuis/Hôpital: 009 - ZIEKENHUISNETWERK AN...,Antwerpen/Anvers,Vlaanderen/Flandre,PAANTW01A,117,11704,ZIEKENHUISNETWERK ANTWERPEN (ZNA) ZNA STUIVENBERG,"(51.2223214, 4.4350678)"
3,"110001: PIT Lier, Heilig Hart",Ziekenhuis/Hôpital: 097 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PALIER01A,171,17101,HEILIG HART ZIEKENHUIS HEILIG HART,"(50.8723845, 4.699762199999999)"
4,113001: PIT HEILIG HART ZIEKENHUIS,Ziekenhuis/Hôpital: 102 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PAMOL_01A,190,19001,HEILIG HART ZIEKENHUIS HEILIG HART ZIEKENHUIS MOL,"(51.1870839, 5.11425)"
5,200001: PIT LOKEREN,Ziekenhuis/Hôpital: 595 - VITAZ --- Campus/Sit...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,POLOKE01A,440,44001,VITAZ VITAZ CAMPUS LOKEREN,"(51.1040179, 3.9895048)"
6,"210001: PIT ASSE, OLV",Ziekenhuis/Hôpital: 126 - ONZE-LIEVE-VROUW ZIE...,Vlaams Brabant/Brabant flamand,Vlaanderen/Flandre,PVASSE01A,480,48001,ONZE OLV ZIEKENHUIS ASSE,"(50.9097128, 4.1955184)"
7,301001: PIT ST Jan (Associatie St Lucas),Ziekenhuis/Hôpital: 049 - AZ SINT-JAN BRUGGE -...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBRUG01A,391,39102,AZ SINT AZ SINT,"(34.0489281, -111.0937311)"
8,301001: PIT St Lucas Brugge (associatie St Jan),Ziekenhuis/Hôpital: 140 - AZ SINT-LUCAS --- Ca...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBRUG02A,391,39101,AZ SINT SINT,"(34.0489281, -111.0937311)"
9,303001: PIT Blankenberge,Ziekenhuis/Hôpital: 392 - AZ ZENO --- Campus/S...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBLAN01A,374,37401,AZ ZENO AZ ZENO BLANKENBERGE,"(51.327595, 3.2912258)"


In [150]:
pit_loc[pit_loc['pit_hospital_coordinates'].isna()]

,unit,campus,province,region,unit_id,ambucode,ambusitecode,pit_hospital,pit_hospital_coordinates
10,"404001: PIT Genk, St Jan",Ziekenhuis/Hôpital: 371 - ZIEKENHUIS OOST-LIMB...,Limburg/Limbourg,Vlaanderen/Flandre,PLGENK01A,776,77601,ZIEKENHUIS OOST SINT,None
13,523001: PIT St Pierre Bruxelles,Ziekenhuis/Hôpital: 076 - CHU SAINT-PIERRE ---...,None,Brussels Hoofdstedelijk Gewest\nRégion de Brux...,PBBRUX01A,280,28001,CHU SAINT PORTE DE HAL,None
21,809001: PIT MALMEDY,Ziekenhuis/Hôpital: 004 - CENTRE HOSPITALIER R...,Luik/Liège,Wallonië/Wallonie,PGMALM01A,637,63701,CENTRE HOSPITALIER REINE ASTRID REINE ASTRID,None
23,903001: PIT LIBRAMONT,Ziekenhuis/Hôpital: 168 - VIVALIA - CENTRE HOS...,Luxemburg/Luxembourg,Wallonië/Wallonie,PXBOUI01A,870,87005,VIVALIA HOPITAL DE LIBRAMONT,None


In [152]:
print(pit_loc[pit_loc['pit_hospital_coordinates'].isna()]["campus"].tolist())

['Ziekenhuis/Hôpital: 371 - ZIEKENHUIS OOST-LIMBURG --- Campus/Site: 3560 - SINT-JAN GENK', 'Ziekenhuis/Hôpital: 076 - CHU SAINT-PIERRE --- Campus/Site: 1520 - PORTE DE HAL', 'Ziekenhuis/Hôpital: 004 - CENTRE HOSPITALIER REINE ASTRID --- Campus/Site: 2950 - REINE ASTRID', 'Ziekenhuis/Hôpital: 168 - VIVALIA - CENTRE HOSPITALIER CENTRE ARDENNE --- Campus/Site: 3240 - HOPITAL DE LIBRAMONT']


In [157]:
pit_loc.at[10, "pit_hospital_coordinates"] = geocode("ZIEKENHUIS OOST-LIMBURG SINT-JAN GENK")

In [160]:
pit_loc.at[13, "pit_hospital_coordinates"] = geocode("CHU SAINT-PIERRE")
pit_loc.at[21, "pit_hospital_coordinates"] = geocode("CENTRE HOSPITALIER REINE ASTRIDL")

In [ ]:
pit_loc.at[23, "pit_hospital_coordinates"] = geocode("CENTRE HOSPITALIER CENTRE ARDENNE HOPITAL DE LIBRAMONT")

In [161]:
pit_loc

,unit,campus,province,region,unit_id,ambucode,ambusitecode,pit_hospital,pit_hospital_coordinates
0,"100001: PIT Rumst, AZ Rivierland",Ziekenhuis/Hôpital: 104 - AZ RIVIERENLAND --- ...,Antwerpen/Anvers,Vlaanderen/Flandre,PARUMS01A,191,19101,AZ RIVIERENLAND RUMST,"(51.1064835, 4.371312)"
1,"102001: PIT Deinze, St Vincentius",Ziekenhuis/Hôpital: 134 - AZ SINT-VINCENTIUS -...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,PODEIN01A,413,41301,AZ SINT AZ SINT,"(34.0489281, -111.0937311)"
2,102101: PIT Stuivenberg,Ziekenhuis/Hôpital: 009 - ZIEKENHUISNETWERK AN...,Antwerpen/Anvers,Vlaanderen/Flandre,PAANTW01A,117,11704,ZIEKENHUISNETWERK ANTWERPEN (ZNA) ZNA STUIVENBERG,"(51.2223214, 4.4350678)"
3,"110001: PIT Lier, Heilig Hart",Ziekenhuis/Hôpital: 097 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PALIER01A,171,17101,HEILIG HART ZIEKENHUIS HEILIG HART,"(50.8723845, 4.699762199999999)"
4,113001: PIT HEILIG HART ZIEKENHUIS,Ziekenhuis/Hôpital: 102 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PAMOL_01A,190,19001,HEILIG HART ZIEKENHUIS HEILIG HART ZIEKENHUIS MOL,"(51.1870839, 5.11425)"
5,200001: PIT LOKEREN,Ziekenhuis/Hôpital: 595 - VITAZ --- Campus/Sit...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,POLOKE01A,440,44001,VITAZ VITAZ CAMPUS LOKEREN,"(51.1040179, 3.9895048)"
6,"210001: PIT ASSE, OLV",Ziekenhuis/Hôpital: 126 - ONZE-LIEVE-VROUW ZIE...,Vlaams Brabant/Brabant flamand,Vlaanderen/Flandre,PVASSE01A,480,48001,ONZE OLV ZIEKENHUIS ASSE,"(50.9097128, 4.1955184)"
7,301001: PIT ST Jan (Associatie St Lucas),Ziekenhuis/Hôpital: 049 - AZ SINT-JAN BRUGGE -...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBRUG01A,391,39102,AZ SINT AZ SINT,"(34.0489281, -111.0937311)"
8,301001: PIT St Lucas Brugge (associatie St Jan),Ziekenhuis/Hôpital: 140 - AZ SINT-LUCAS --- Ca...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBRUG02A,391,39101,AZ SINT SINT,"(34.0489281, -111.0937311)"
9,303001: PIT Blankenberge,Ziekenhuis/Hôpital: 392 - AZ ZENO --- Campus/S...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBLAN01A,374,37401,AZ ZENO AZ ZENO BLANKENBERGE,"(51.327595, 3.2912258)"


In [170]:
pit_loc[pit_loc.duplicated()]

,unit,campus,province,region,unit_id,ambucode,ambusitecode,pit_hospital,pit_hospital_coordinates


## Save and read the dataset

In [162]:
pit_loc.to_csv('C:\KU Leuven\Modern Data Analytics - MDA\MDA Project\PIT_coordinates_new.csv', index=False)

In [163]:
pit_coord = pd.read_csv(r'C:\KU Leuven\Modern Data Analytics - MDA\MDA Project\PIT_coordinates_new.csv', index_col=False)

In [164]:
pit_coord

,unit,campus,province,region,unit_id,ambucode,ambusitecode,pit_hospital,pit_hospital_coordinates
0,"100001: PIT Rumst, AZ Rivierland",Ziekenhuis/Hôpital: 104 - AZ RIVIERENLAND --- ...,Antwerpen/Anvers,Vlaanderen/Flandre,PARUMS01A,191,19101,AZ RIVIERENLAND RUMST,"(51.1064835, 4.371312)"
1,"102001: PIT Deinze, St Vincentius",Ziekenhuis/Hôpital: 134 - AZ SINT-VINCENTIUS -...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,PODEIN01A,413,41301,AZ SINT AZ SINT,"(34.0489281, -111.0937311)"
2,102101: PIT Stuivenberg,Ziekenhuis/Hôpital: 009 - ZIEKENHUISNETWERK AN...,Antwerpen/Anvers,Vlaanderen/Flandre,PAANTW01A,117,11704,ZIEKENHUISNETWERK ANTWERPEN (ZNA) ZNA STUIVENBERG,"(51.2223214, 4.4350678)"
3,"110001: PIT Lier, Heilig Hart",Ziekenhuis/Hôpital: 097 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PALIER01A,171,17101,HEILIG HART ZIEKENHUIS HEILIG HART,"(50.8723845, 4.699762199999999)"
4,113001: PIT HEILIG HART ZIEKENHUIS,Ziekenhuis/Hôpital: 102 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PAMOL_01A,190,19001,HEILIG HART ZIEKENHUIS HEILIG HART ZIEKENHUIS MOL,"(51.1870839, 5.11425)"
5,200001: PIT LOKEREN,Ziekenhuis/Hôpital: 595 - VITAZ --- Campus/Sit...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,POLOKE01A,440,44001,VITAZ VITAZ CAMPUS LOKEREN,"(51.1040179, 3.9895048)"
6,"210001: PIT ASSE, OLV",Ziekenhuis/Hôpital: 126 - ONZE-LIEVE-VROUW ZIE...,Vlaams Brabant/Brabant flamand,Vlaanderen/Flandre,PVASSE01A,480,48001,ONZE OLV ZIEKENHUIS ASSE,"(50.9097128, 4.1955184)"
7,301001: PIT ST Jan (Associatie St Lucas),Ziekenhuis/Hôpital: 049 - AZ SINT-JAN BRUGGE -...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBRUG01A,391,39102,AZ SINT AZ SINT,"(34.0489281, -111.0937311)"
8,301001: PIT St Lucas Brugge (associatie St Jan),Ziekenhuis/Hôpital: 140 - AZ SINT-LUCAS --- Ca...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBRUG02A,391,39101,AZ SINT SINT,"(34.0489281, -111.0937311)"
9,303001: PIT Blankenberge,Ziekenhuis/Hôpital: 392 - AZ ZENO --- Campus/S...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBLAN01A,374,37401,AZ ZENO AZ ZENO BLANKENBERGE,"(51.327595, 3.2912258)"


## REMARK !!! THE PIT Dataset needs to be explored for duplicates